#### You will implement value iteration and policy iteration for the Frozen Lake environment from OpenAI Gym.

# import needed libraries and functions

In [3]:
import numpy as np
import gym
import time
from lake_envs import *
from utils import render_single
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)

## ! Attention: read the following to get familiar with the parameters of the functions that you will implement

In [134]:
'''
For policy_evaluation, policy_improvement, policy_iteration and value_iteration,
the parameters P, nS, nA, gamma are defined as follows:

	P: nested dictionary
		From gym.core.Environment
		For each pair of states in [0, nS-1] and actions in [0, nA-1], P[state][action] is 
        (a list of) tuples of the form (probability, nextstate, reward, terminal) where
			- probability: float
				the probability of transitioning from "state" to "nextstate" with "action"
			- nextstate: int
				denotes the state we transition to (in range [0, nS - 1])
			- reward: int
				either 0 or 1, the reward for transitioning from "state" to
				"nextstate" with "action"
			- terminal: bool
			  True when "nextstate" is a terminal state (hole or goal), False otherwise
	nS: int
		number of states in the environment
	nA: int
		number of actions in the environment
	gamma: float
		Discount factor. Number in range [0, 1)
'''

'\nFor policy_evaluation, policy_improvement, policy_iteration and value_iteration,\nthe parameters P, nS, nA, gamma are defined as follows:\n\n\tP: nested dictionary\n\t\tFrom gym.core.Environment\n\t\tFor each pair of states in [0, nS-1] and actions in [0, nA-1], P[state][action] is \n        (a list of) tuples of the form (probability, nextstate, reward, terminal) where\n\t\t\t- probability: float\n\t\t\t\tthe probability of transitioning from "state" to "nextstate" with "action"\n\t\t\t- nextstate: int\n\t\t\t\tdenotes the state we transition to (in range [0, nS - 1])\n\t\t\t- reward: int\n\t\t\t\teither 0 or 1, the reward for transitioning from "state" to\n\t\t\t\t"nextstate" with "action"\n\t\t\t- terminal: bool\n\t\t\t  True when "nextstate" is a terminal state (hole or goal), False otherwise\n\tnS: int\n\t\tnumber of states in the environment\n\tnA: int\n\t\tnumber of actions in the environment\n\tgamma: float\n\t\tDiscount factor. Number in range [0, 1)\n'

# (a) Implement policy iteration [10 pts]

## implement policy evaluation

In [135]:
def policy_evaluation(P, nS, nA, policy, gamma=0.9, tol=1e-3):
	"""Evaluate the value function from a given policy.

	Parameters
	----------
	P, nS, nA, gamma:
		defined at beginning of file
	policy: np.array[nS]
		The policy to evaluate. Maps states to actions.
	tol: float
		Terminate policy evaluation when
			max |value_function(s) - prev_value_function(s)| < tol
	Returns
	-------
	value_function: np.ndarray[nS]
		The value function of the given policy, where value_function[s] is
		the value of state s
	"""

	value_function = np.zeros(nS)

	############################
	count = 0
	while True:
		new_value_function = np.zeros(nS)

		for state in range(nS):
			action = policy[state]
			for probability, nextstate, reward, terminal in P[state][action]:
				new_value_function[state] += probability * (reward + gamma * value_function[nextstate])
		
		if np.max(np.abs(new_value_function - value_function)) < tol:
			print("Policy Evaluation converged in {} iterations.".format(count))
			break
		else:
			count += 1
			value_function = new_value_function

	############################
	return value_function

## implement policy improvement

In [136]:
def policy_improvement(P, nS, nA, value_from_policy, policy, gamma=0.9):
	"""Given the value function from policy improve the policy.

	Parameters
	----------
	P, nS, nA, gamma:
		defined at beginning of file
	value_from_policy: np.ndarray
		The value calculated from the policy
	policy: np.array
		The previous policy.

	Returns
	-------
	new_policy: np.ndarray[nS]
		An array of integers. Each integer is the optimal action to take
		in that state according to the environment dynamics and the
		given value function.
	"""

	new_policy = np.zeros(nS, dtype='int')

	############################
	# for state in P:
	# 	for i, action in enumerate(state):
	# 		expectations = []
	# 		# sum up probability-weighted sum of values for that state-action pair
	# 		for probability, nextstate, reward, terminal in action:
	# 			expectations[i] += probability * (reward + value_from_policy[nextstate])
	# 			new_policy[state] = np.argmax(expectations)

	for s in range(nS):
		# choose the action that maximizes the expected value
		new_policy[s] = np.argmax([
			sum(prob * (reward + gamma * value_from_policy[nextstate])
				for prob, nextstate, reward, terminal in P[s][a])
			for a in range(nA)
		])			

	############################
	return new_policy

## implement policy iteration

In [137]:
def policy_iteration(P, nS, nA, gamma=0.9, tol=1e-3):
	"""Runs policy iteration.

	You should call the policy_evaluation() and policy_improvement() methods to
	implement this method.

	Parameters
	----------
	P, nS, nA, gamma:
		defined at beginning of file
	tol: float
		tol parameter used in policy_evaluation()
	Returns:
	----------
	value_function: np.ndarray[nS]
	policy: np.ndarray[nS]
	"""

	value_function = np.zeros(nS)
	policy = np.zeros(nS, dtype=int)

	############################
	count = 0
	while True:
		policy = policy_improvement(P, nS, nA, value_function, policy, gamma=0.9)
		new_value_function = policy_evaluation(P, nS, nA, policy, gamma, tol)
		if np.max(np.abs(value_function-new_value_function)) < tol:
			print("Policy Iteration converged in {} iterations.".format(count))
			break
		else:
			count += 1
			value_function = new_value_function
					
	############################
	return value_function, policy

## run policy iteration on 'Deterministic-4x4-FrozenLake-v0'

In [138]:
env = gym.make("Deterministic-4x4-FrozenLake-v0")
V_pi, p_pi = policy_iteration(env.P, env.nS, env.nA, gamma=0.9, tol=1e-3)

Policy Evaluation converged in 1 iterations.
Policy Evaluation converged in 2 iterations.
Policy Evaluation converged in 3 iterations.
Policy Evaluation converged in 5 iterations.
Policy Evaluation converged in 5 iterations.
Policy Evaluation converged in 6 iterations.
Policy Evaluation converged in 6 iterations.
Policy Iteration converged in 6 iterations.


d:\Installs\Anaconda3\envs\drl\Lib\site-packages\gym\envs\registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


In [139]:
# print the final value function and policy
print('Final value function:\n',V_pi.reshape(4,4))
print('\nFinal policy:\n',p_pi.reshape(4,4))

Final value function:
 [[0.59049 0.6561  0.729   0.6561 ]
 [0.6561  0.      0.81    0.     ]
 [0.729   0.81    0.9     0.     ]
 [0.      0.9     1.      0.     ]]

Final policy:
 [[1 2 1 0]
 [1 0 1 0]
 [2 1 1 0]
 [0 2 2 0]]


In [140]:
# test the policy in the environment
render_single(env, p_pi, 100)


SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
Episode reward: 1.000000


## run policy iteration on 'Stochastic-4x4-FrozenLake-v0'

In [141]:
env = gym.make("Stochastic-4x4-FrozenLake-v0")
V_pi, p_pi = policy_iteration(env.P, env.nS, env.nA, gamma=0.9, tol=1e-3)

Policy Evaluation converged in 8 iterations.
Policy Evaluation converged in 18 iterations.
Policy Evaluation converged in 20 iterations.
Policy Evaluation converged in 26 iterations.
Policy Evaluation converged in 26 iterations.
Policy Evaluation converged in 26 iterations.
Policy Iteration converged in 5 iterations.


In [142]:
# print the final value function and policy
print('Final value function:\n',V_pi.reshape(4,4))
print('\nFinal policy:\n',p_pi.reshape(4,4))

Final value function:
 [[0.06144365 0.05515526 0.06984368 0.05087275]
 [0.08503352 0.         0.10970645 0.        ]
 [0.13983835 0.24364732 0.29690307 0.        ]
 [0.         0.37709623 0.63751037 0.        ]]

Final policy:
 [[0 3 0 3]
 [0 0 0 0]
 [3 1 0 0]
 [0 2 1 0]]


In [146]:
# test the policy in the environment
render_single(env, p_pi, 100)


SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
Episode reward: 1.000000


# (b) Implement value iteration [10 pts]

## implement value iteration

In [25]:
def value_iteration(P, nS, nA, gamma=0.9, tol=1e-3):
	"""
	Learn value function and policy by using value iteration method for a given
	gamma and environment.

	Parameters:
	----------
	P, nS, nA, gamma:
		defined at beginning of file
	tol: float
		Terminate value iteration when
			max |value_function(s) - prev_value_function(s)| < tol
	Returns:
	----------
	value_function: np.ndarray[nS]
	policy: np.ndarray[nS]
	"""

	value_function = np.zeros(nS)
	policy = np.zeros(nS, dtype=int)
	############################
	new_value_function = np.zeros(nS)
	counter = 0
	while True:
		for s in range(nS):
			new_value_function[s] = np.max([
				sum(probability * (reward + gamma * value_function[nextstate])
				for probability, nextstate, reward, _ in P[s][a]) 
				for a in range(nA)
			])
		if np.max(np.abs(new_value_function - value_function)) < tol:
			print("Value Iteration converged in {} iterations.".format(counter))
			break
		else:
			value_function = new_value_function.copy()
			counter += 1
	for s in range(nS):
		policy[s] = np.argmax([
			sum(probability*(reward + gamma * value_function[nextstate])
	   		for probability, nextstate, reward, _ in P[s][a])
			for a in range(nA)
		])
	############################
	return value_function, policy

## run value iteration on 'Deterministic-4x4-FrozenLake-v0'

In [26]:
env = gym.make("Deterministic-4x4-FrozenLake-v0")
V_vi, p_vi = value_iteration(env.P, env.nS, env.nA, gamma=0.9, tol=1e-3)

Value Iteration converged in 6 iterations.


In [27]:
# print the final value function and policy
print('Final value function:\n',V_vi.reshape(4,4))
print('\nFinal policy:\n',p_vi.reshape(4,4))

Final value function:
 [[0.59049 0.6561  0.729   0.6561 ]
 [0.6561  0.      0.81    0.     ]
 [0.729   0.81    0.9     0.     ]
 [0.      0.9     1.      0.     ]]

Final policy:
 [[1 2 1 0]
 [1 0 1 0]
 [2 1 1 0]
 [0 2 2 0]]


In [28]:
# test the policy in the environment
render_single(env, p_vi, 100)


SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
Episode reward: 1.000000


## run value iteration on 'Stochastic-4x4-FrozenLake-v0'

In [29]:
env = gym.make("Stochastic-4x4-FrozenLake-v0")
V_vi, p_vi = value_iteration(env.P, env.nS, env.nA, gamma=0.9, tol=1e-3)

Value Iteration converged in 26 iterations.


In [30]:
# print the final value function and policy
print('Final value function:\n',V_vi.reshape(4,4))
print('\nFinal policy:\n',p_vi.reshape(4,4))

Final value function:
 [[0.06162274 0.05531399 0.06996222 0.05101703]
 [0.08519461 0.         0.10976852 0.        ]
 [0.13996615 0.2437311  0.29696299 0.        ]
 [0.         0.37715398 0.63753958 0.        ]]

Final policy:
 [[0 3 0 3]
 [0 0 0 0]
 [3 1 0 0]
 [0 2 1 0]]


In [31]:
# test the policy in the environment
render_single(env, p_vi, 100)


SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  

# (c) Answer questions [5 pts]

### You have run both methods on the Deterministic-4x4-FrozenLake-v0 and Stochastic-4x4-FrozenLake-v0 environments. In the second environment, the dynamics of the world are stochastic. How does stochasticity affect the number of iterations required, and the resulting policy? Write your answer below.

Policies in stochastic environments prioritize avoiding holes over reaching the reward quickly. Hence, it takes longer to converge.